# <span style="font-family:cursive;text-align:center"><font color=red> End-to-End Wine Quality Prediction using Kfp pipelines(Kubeflow SDK) </span>

In [ ]:
from kfp import components
from kfp import dsl
from typing import List
import kfp
from typing import NamedTuple
from kfp.components import func_to_container_op, create_component_from_func

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'> Read and save data </font></h3>

This method loads the data from the load_wine from sklearn.datasets and saves it to /kfs_private as train.csv and test.csv and @create_component_from_func decorator is used to conver this function into a component

Note: If you have a minio standalone or minio file-storage attached to your cluster. You can directly specify the storage path as /kfs_private or /kfs_public and further bucket-names

In [ ]:
@create_component_from_func
def data_loading_splitting():
    import os
    os.system("pip install pandas scikit-learn")
    import pandas as pd
    from sklearn.datasets import load_wine
    from sklearn.model_selection import train_test_split
    x, y = load_wine(return_X_y=True)
    x = pd.DataFrame(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    x_train['y'] = y_train
    x_test['y'] = y_test

    x_train.to_csv('/kfs_private/train.csv', index=False)
    x_test.to_csv('kfs_private/test.csv', index=False)

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'> Modelling using Decision tree </font></h3>

This method reads train.csv and test.csv from /kfs_private/data_dir and creating a DecisionTreeClassifier object and printing the accuracy score

In [ ]:
@create_component_from_func # This is another way to create component from function, to use create_component_from_func as a decorator
def decision_tree():
    import os
    os.system("pip install pandas scikit-learn")
    import pandas as pd
    from sklearn.tree import DecisionTreeClassifier
    train = pd.read_csv('/kfs_private/train.csv')
    test = pd.read_csv('/kfs_private/test.csv')
    x_train, y_train = train.drop(columns=['y'], axis=1), train['y']
    x_test, y_test = test.drop(columns=['y'], axis=1), test['y']
    clf = DecisionTreeClassifier()
    clf.fit(x_train, y_train)
    print(clf.score(x_test, y_test))

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'> Modelling using Logistic regression </font></h3>

This method reads train.csv and test.csv from /kfs_private/data_dir and creating a Logistic Regression object and printing the accuracy score

In [ ]:
@create_component_from_func
def logistic_regression():
    import os
    os.system("pip install pandas scikit-learn")
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    train = pd.read_csv('/kfs_private/train.csv')
    test = pd.read_csv('/kfs_private/test.csv')
    x_train, y_train = train.drop(columns=['y'], axis=1), train['y']
    x_test, y_test = test.drop(columns=['y'], axis=1), test['y']
    clf = LogisticRegression()
    clf.fit(x_train, y_train)
    print(clf.score(x_test, y_test))

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'> Modelling using Random forest </font></h3>

This method reads train.csv and test.csv from /kfs_private/data_dir and creating a RandomForestClassifier object and printing the accuracy score

In [ ]:
@create_component_from_func
def random_forest():
    import os
    os.system("pip install pandas scikit-learn")
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    train = pd.read_csv('/kfs_private/train.csv')
    test = pd.read_csv('/kfs_private/test.csv')
    x_train, y_train = train.drop(columns=['y'], axis=1), train['y']
    x_test, y_test = test.drop(columns=['y'], axis=1), test['y']
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    print(clf.score(x_test, y_test))

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'> Exit handler </font></h3>
    
This method is going to be used as a exit handler for the pipeline

that means this method is going to be executed at the end of the pipeline regardless, wheather pipeline executes successfully or not

In [ ]:
@create_component_from_func
def echo_msg(msg: str):
    """Echo a message by parameter."""
    print(msg)